Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

## Build Spark Session

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



Let's work with train dataset:

**Confirm if this is a dataframe or not:**

**Show 5 rows.**

**Display schema for the dataset:**

**Statistical summary:**

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

**Display your result:**

**Can you display your answer in ratio form?(Hint: Use UDF.)**






**Can you get the number of males and females?**


**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column.)

**Create temporary view PySpark:**

**How many people survived, and how many didn't survive? By SQL:**

**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column.)

**Can you do this via SQL?**

**Display a ratio for p-class:**


**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



**Display count:**

**Temporary view PySpark:**

**Can you define the number of null values in each column?**


**Create Dataframe for null values**

1. Column
2. Number of missing values.

## Preprocessing 

**Can you show me the name column from your temporary table?**

**Run this code:**

In [ ]:
combined = combined.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display the title and count "Title" column:**

**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

**Run the function:**

In [ ]:
def impute_title(title):
    return titles_map[title]

**Apply the function on "Title" column using UDF:**

**Display "Title" from table and group by "Title" column:**

## **Preprocessing Age**

**Based on the age mean, you will fill in the missing age values:**

**Fill missing age with age mean:**

## **Preprocessing Embarked**

**Select Embarked, count them, order by count Desc, and save in grouped_Embarked variable:**




**Show groupped_Embarked:**

**Get the groupped_Embarked:** 

**Fill missing values with Top 'S' of grouped_Embarked:**

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



**Show the result:**

**Create the temporary view:**

**Select "Cabin" column, count Cabin column, Group by "Cabin" column, Order By count DESC**  

**Fill missing values with "U":**

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

____________________________________________

**Use Pipline to fit and transform:**

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None) A feature transformer that merges multiple columns into a vector column.**



**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
